# Chapter 02: Fine-tuning 유형 및 방법론

## 1. 학습 목표

* Full, Partial, PEFT Fine-tuning의 차이점과 장단점을 이해한다.
* LoRA, QLoRA, DoRA의 원리를 파악하고 구현 방법을 익힌다.
* 상황에 맞는 최적의 Fine-tuning 전략을 선택할 수 있다.
* gpt-oss-20b 모델에 적합한 학습 설정을 구성한다.

## 2. Fine-tuning 유형 분류

Fine-tuning은 **학습 대상 가중치**의 범위에 따라 크게 세 가지 유형으로 분류된다.

```text
Fine-tuning 유형
├── Full Fine-tuning: 모든 가중치(100%) 학습
├── Partial Tuning: 일부 레이어(10~50%) 학습
└── PEFT (Parameter-Efficient): 새로운 소규모 가중치(0.1~1%) 학습

```

### 2.1 유형별 비교

| 특성 | Full Fine-tuning | Partial Tuning | PEFT (LoRA 등) |
| --- | --- | --- | --- |
| **학습 파라미터** | 전체 (100%) | 일부 (10-50%) | 극소수 (0.1-1%) |
| **VRAM 요구량** | 매우 높음 (모델 크기의 4배 이상) | 높음 | 낮음 (모델 크기와 유사) |
| **성능 잠재력** | 최고 | 중상 | 높음 (Full 대비 95-99%) |
| **망각 위험** | 높음 | 중간 | 낮음 |
| **권장도 (2025)** | 낮음 (연구 목적) | 낮음 | **매우 높음 (표준)** |

## 3. Full Fine-tuning

### 3.1 개념

모델의 **모든 파라미터**를 학습 데이터로 업데이트하는 방식이다. 가장 전통적인 방식이지만, 모델 크기가 커짐에 따라 기하급수적으로 늘어나는 VRAM 요구량으로 인해 대형 모델에서는 거의 사용되지 않는다.

### 3.2 구현 코드 (개념적)

In [1]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM

# 모델 로드
model = AutoModelForCausalLM.from_pretrained("openai/gpt-oss-20b")

# Full Fine-tuning 설정: 모든 파라미터의 그래디언트 계산 활성화
for param in model.parameters():
    param.requires_grad = True

print(f"전체 학습 가능 파라미터 수: {sum(p.numel() for p in model.parameters())}")

/home/josh/anaconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
MXFP4 quantization requires Triton and kernels installed: CUDA requires Triton >= 3.4.0, XPU requires Triton >= 3.5.0, we will default to dequantizing the model to bf16
Loading checkpoint shards: 100%|██████████| 3/3 [00:48<00:00, 16.07s/it]


전체 학습 가능 파라미터 수: 20914757184


## 4. Parameter-Efficient Fine-tuning (PEFT)

2025년 현재, LLM Fine-tuning의 표준은 PEFT다. 기존 가중치를 동결(Freeze)하고 **새로운 소규모 학습 가능한 파라미터**를 추가하거나, 특정 부분만 학습하여 메모리 효율성과 성능을 동시에 잡는다.

### 4.1 LoRA (Low-Rank Adaptation)

#### 핵심 원리

거대 언어 모델의 가중치 행렬은 "Intrinsic Rank(본질적 차원)"가 낮다는 가설에 기반한다. 즉, 전체 가중치를 다 바꿀 필요 없이, **저순위(Low-Rank) 분해**된 작은 행렬 두 개(, )만 학습해도 충분한 변화를 줄 수 있다.

```text
수식: W' = W + ΔW = W + (A × B)
- W: 원본 가중치 (동결)
- A: d × r 행렬 (학습)
- B: r × d 행렬 (학습)
- r: Rank (매우 작은 값, 예: 8, 16)

```

#### 파라미터 절감 효과

4096 × 4096 크기의 가중치 행렬을 예로 들면:

* **Full Fine-tuning**: 16,777,216개 파라미터 학습
* **LoRA (r=8)**: (4096×8) + (8×4096) = 65,536개 파라미터 학습
* **결과**: 약 **256배**의 파라미터 절감 효과가 있다.

#### LoRA 레이어 구현 예시

LoRA의 작동 원리를 이해하기 위한 간단한 PyTorch 구현이다.

In [2]:
class LoRALayer(nn.Module):
    """
    LoRA 레이어의 원리를 보여주는 간단한 구현이다.
    원본 가중치는 동결하고 저순위 행렬 A, B만 학습한다.
    """
    def __init__(self, original_layer, rank=8, alpha=16):
        super().__init__()
        self.original_layer = original_layer

        # 원본 레이어 동결 (핵심)
        for param in self.original_layer.parameters():
            param.requires_grad = False

        # 입력/출력 차원 획득
        in_features = original_layer.in_features
        out_features = original_layer.out_features

        # LoRA 행렬 A, B 초기화
        # A는 정규분포, B는 0으로 초기화하여 학습 시작 시 원본과 동일한 출력을 보장한다.
        self.lora_A = nn.Parameter(torch.randn(in_features, rank) * 0.01)
        self.lora_B = nn.Parameter(torch.zeros(rank, out_features))

        # 스케일링 팩터 (학습률과 유사한 역할)
        self.scaling = alpha / rank

    def forward(self, x):
        # 1. 원본 경로 (Frozen)
        original_output = self.original_layer(x)

        # 2. LoRA 경로 (Trainable): x @ A @ B
        lora_output = (x @ self.lora_A @ self.lora_B) * self.scaling

        # 3. 최종 합산
        return original_output + lora_output

# 사용 예시
original = nn.Linear(4096, 4096)
lora_layer = LoRALayer(original, rank=8, alpha=16)

trainable = sum(p.numel() for p in lora_layer.parameters() if p.requires_grad)
total = sum(p.numel() for p in lora_layer.parameters())

print(f"학습 가능 파라미터: {trainable:,}")
print(f"전체 파라미터: {total:,}")
print(f"학습 비율: {trainable/total*100:.4f}%")

학습 가능 파라미터: 65,536
전체 파라미터: 16,846,848
학습 비율: 0.3890%


### 4.2 QLoRA (Quantized LoRA)

#### 개념

LoRA에 **4-bit 양자화(Quantization)**를 결합하여 메모리 사용량을 극단적으로 줄인 방식이다. 소비자용 GPU(RTX 3090/4090)에서도 거대 모델을 학습할 수 있게 해주는 핵심 기술이다.

#### 핵심 기술 3가지

1. **4-bit NormalFloat (NF4)**: 정규분포를 따르는 가중치에 최적화된 새로운 데이터 타입이다. 정보 손실을 최소화한다.
2. **Double Quantization**: 양자화에 사용되는 상수(Quantization Constant)조차 양자화하여 추가적인 메모리를 절약한다.
3. **Paged Optimizers**: GPU 메모리가 부족할 때 CPU RAM을 자동으로 활용하여 OOM(Out of Memory) 오류를 방지한다.

#### 메모리 비교 (20B 모델 기준)

| 방식 | 모델 로드 | 학습 오버헤드 | 총 필요 VRAM |
| --- | --- | --- | --- |
| Full (FP16) | 40 GB | 120 GB+ | 160 GB+ (A100 4장) |
| LoRA (FP16) | 40 GB | 2 GB | 45 GB (A100 1장) |
| **QLoRA (4-bit)** | **10 GB** | **2 GB** | **14~16 GB (RTX 4090)** |

### 4.3 DoRA (Weight-Decomposed LoRA)

2024년에 발표된 LoRA의 개선판이다. 가중치를 **크기(Magnitude)**와 **방향(Direction)**으로 분해하여 학습한다.

* **원리**:
*  (크기): 벡터로 직접 학습
*  (방향): LoRA로 학습


* **장점**: LoRA보다 학습 안정성이 높고, 낮은 Rank에서도 Full Fine-tuning에 가까운 성능을 낸다.
* **사용법**: `peft` 라이브러리에서 `use_dora=True` 옵션만 켜면 된다.

## 5. 학습 방법 선택 가이드

상황에 따라 어떤 방법을 선택해야 할지 결정하는 가이드다.

In [3]:
def recommend_finetuning_method(gpu_vram_gb, model_size_b, goal):
    """
    하드웨어와 목적에 따른 Fine-tuning 방식을 추천하는 함수다.
    """
    print(f"분석 조건: VRAM {gpu_vram_gb}GB, 모델 {model_size_b}B, 목표 '{goal}'")

    # 1. 메모리 제약 확인
    # 4-bit 모델 크기(GB) ≈ 파라미터 수 / 2
    required_vram_qlora = (model_size_b / 2) * 1.5

    if gpu_vram_gb < required_vram_qlora:
        print("결과: 학습 불가능. 더 높은 VRAM의 GPU가 필요하다.")
        return

    # 2. 방식 추천
    if goal == "최고 성능" and gpu_vram_gb > model_size_b * 8:
        print("추천: Full Fine-tuning (자원이 충분함)")
    elif goal == "일반적인 목적" or goal == "가성비":
        print("추천: QLoRA + SFT")
        print("  - 가장 효율적이며 성능 저하가 거의 없다.")
        print("  - bitsandbytes 라이브러리 활용 필수.")
    elif goal == "성능 최적화" and "메모리 여유" in goal:
        print("추천: DoRA (Weight-Decomposed LoRA)")
        print("  - LoRA보다 높은 학습 안정성과 성능을 제공한다.")
    elif goal == "추론 속도":
        print("추천: LoRA/QLoRA 학습 후 병합(Merge)")
        print("  - 학습된 어댑터를 베이스 모델에 합쳐 단일 모델로 배포한다.")

# gpt-oss-20b 기준 추천 시뮬레이션
recommend_finetuning_method(gpu_vram_gb=24, model_size_b=20, goal="가성비")

분석 조건: VRAM 24GB, 모델 20B, 목표 '가성비'
추천: QLoRA + SFT
  - 가장 효율적이며 성능 저하가 거의 없다.
  - bitsandbytes 라이브러리 활용 필수.


## 6. gpt-oss-20b를 위한 권장 설정

gpt-oss-20b 모델을 RTX 3090/4090급(24GB VRAM) 환경에서 학습하기 위한 최적의 설정을 정리한다.

In [4]:
# gpt-oss-20b QLoRA 권장 설정
recommended_config = {
    # 1. 모델 양자화 설정 (QLoRA 핵심)
    "quantization": {
        "load_in_4bit": True,
        "bnb_4bit_quant_type": "nf4",           # NormalFloat 4-bit
        "bnb_4bit_compute_dtype": "bfloat16",   # 연산 정밀도
        "bnb_4bit_use_double_quant": True       # 이중 양자화
    },

    # 2. LoRA 설정
    "lora": {
        "r": 16,                    # Rank: 16~32 권장 (복잡한 태스크는 64)
        "lora_alpha": 32,           # Alpha: Rank의 2배
        "target_modules": [         # 모든 Linear 레이어에 적용 권장
            "q_proj", "k_proj", "v_proj", "o_proj",
            "gate_proj", "up_proj", "down_proj"
        ],
        "lora_dropout": 0.05,
        "bias": "none",
        "task_type": "CAUSAL_LM"
    }
}

print("gpt-oss-20b 권장 설정 구성 완료")
for category, conf in recommended_config.items():
    print(f"\n[{category}]")
    for k, v in conf.items():
        print(f"  {k}: {v}")

gpt-oss-20b 권장 설정 구성 완료

[quantization]
  load_in_4bit: True
  bnb_4bit_quant_type: nf4
  bnb_4bit_compute_dtype: bfloat16
  bnb_4bit_use_double_quant: True

[lora]
  r: 16
  lora_alpha: 32
  target_modules: ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']
  lora_dropout: 0.05
  bias: none
  task_type: CAUSAL_LM


## 7. 요약

이번 챕터에서는 Fine-tuning의 다양한 방법론을 비교하고, 현실적인 제약 하에서 최적의 선택지인 **QLoRA**와 최신 기법인 **DoRA**에 대해 학습했다.

* **Full Fine-tuning**은 성능은 좋지만 막대한 자원이 필요하여 실무에서는 제한적으로 사용된다.
* **LoRA**는 가중치 행렬을 저순위 분해하여 파라미터를 획기적으로 줄이는 기술이다.
* **QLoRA**는 4-bit 양자화를 통해 20B급 모델도 소비자용 GPU에서 학습 가능하게 만든다.
* **DoRA**는 크기와 방향을 분리하여 학습 안정성을 높인 LoRA의 발전형이다.

다음 챕터에서는 이러한 학습을 진행하기 위해 가장 중요한 원료인 **데이터셋 준비** 과정을 다룬다.